<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Bispectrum-counter-example" data-toc-modified-id="Bispectrum-counter-example-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Bispectrum counter example</a></span><ul class="toc-item"><li><span><a href="#feature-computation" data-toc-modified-id="feature-computation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>feature computation</a></span><ul class="toc-item"><li><span><a href="#$\rho_{i}^{\otimes-3}$" data-toc-modified-id="$\rho_{i}^{\otimes-3}$-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>$\rho_{i}^{\otimes 3}$</a></span></li><li><span><a href="#$\rho_{i-i_1-i_2}^{\otimes-1}$" data-toc-modified-id="$\rho_{i-i_1-i_2}^{\otimes-1}$-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>$\rho_{i i_1 i_2}^{\otimes 1}$</a></span></li><li><span><a href="#check-degeneracy-of-$\rho_{i}^{\otimes-3}$" data-toc-modified-id="check-degeneracy-of-$\rho_{i}^{\otimes-3}$-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>check degeneracy of $\rho_{i}^{\otimes 3}$</a></span></li></ul></li><li><span><a href="#set-up-an-encoder-decoder-like-architecture" data-toc-modified-id="set-up-an-encoder-decoder-like-architecture-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>set up an encoder-decoder like architecture</a></span></li></ul></li></ul></div>

# Bispectrum counter example

In [4]:
%load_ext autoreload
%autoreload 2 

In [8]:
from ase.io import read,write
from ase.visualize import view
import numpy as np 

from equistore import Labels, TensorBlock, TensorMap
from itertools import product
from utils.clebsh_gordan import ClebschGordanReal

import matplotlib.pyplot as plt

from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from rascal.representations import SphericalExpansion
import copy
import ase 

In [10]:
from utils.mp_utils import *

In [11]:
from generate_bispectrum_structures import generate_nu3_degen_structs, single_partial_derivative

In [13]:
r = np.random.uniform(0.5,2)
z1 = np.random.uniform(0.5,2)
z2 = np.random.uniform(1.5,1.8)
psi = np.random.uniform(0,np.pi)
phi1 = np.random.uniform(np.pi/6, np.pi)
phi2 = np.random.uniform(np.pi/6, np.pi)

parameters = [r, [0, phi1, phi1+phi2], psi, z1, z2, "Li", "Mg", "H"]
single_partial_derivative(parameters, diff_param_names=['r', 'phi', 'psi', 'z1', 'z2'], diff_param='r', delta=1e-6)


(array([[ 0.        ,  0.        ,  0.        ],
        [ 1.        ,  0.        ,  0.        ],
        [ 0.7510827 ,  0.66020813,  0.        ],
        [-0.32385637,  0.94610626,  0.        ],
        [ 0.95596832,  0.29346988,  0.        ],
        [ 0.52426007,  0.85155821,  0.        ],
        [-0.58725012,  0.80940552,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]),
 array([[ 0.        ,  0.        ,  0.        ],
        [ 1.        ,  0.        ,  0.        ],
        [ 0.7510827 ,  0.66020813,  0.        ],
        [-0.32385637,  0.94610626,  0.        ],
        [ 0.95596832,  0.29346988,  0.        ],
        [ 0.52426007,  0.85155821,  0.        ],
        [-0.58725012,  0.80940552,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]))

In [4]:
import pyscf
from pyscf import gto
from pyscf import scf,dft

def pyscf_rks_calc(frame):
    pts = frame.get_positions()
    els = frame.get_chemical_symbols()
    natoms = len(pts)
    atoms = []
    for i in range(natoms):
        coord = pts[i]
        atoms.append([els[i],(coord[0],coord[1],coord[2])])
    mol = gto.M(atom=atoms,symmetry = True)
    mol.basis = "ccpvdz" #"def2-SVP" 
#     mol.basis = {'O': 'ccpvdz', 'H': 'ccpvdz', 'C': 'ccpvdz', 'N':'ccpvdz', 'Ag': 'def2-svp'} 
    mol.charge=0
    mol.spin = 0
    mol.build()
    m = scf.RKS(mol)
#     m = scf.RHF(mol)
    m.xc="pbe"
    m.max_cycle=600
    m.conv_tol=1e-9
    m.conv_tol_grad=1e-8
    m.diis_space=10
#     print(mol.ao_labels())

    return m.kernel() 

def pyscf_rhf_calc(frame):
    pts = frame.get_positions()
    els = frame.get_chemical_symbols()
    natoms = len(pts)
    atoms = []
    for i in range(natoms):
        coord = pts[i]
        atoms.append([els[i],(coord[0],coord[1],coord[2])])
    mol = gto.M(atom=atoms,symmetry = True)
    mol.basis = "ccpvdz" #"def2-SVP" 
#     mol.basis = {'O': 'ccpvdz', 'H': 'ccpvdz', 'C': 'ccpvdz', 'N':'ccpvdz', 'Ag': 'def2-svp'} 
    mol.charge=0
    mol.spin = 0
    mol.build()
#     m = scf.RKS(mol)
    m = scf.RHF(mol)
#     m.xc="pbe"
    m.max_cycle=500
    m.conv_tol=1e-9
#     m.conv_tol_grad=1e-8
    m.diis_space=10
#     print(mol.ao_labels())

    return m.kernel() 

pyscf plugins found in 
/Users/jigyasa/scratch/pyscf
/Users/jigyasa/scratch/./pyscf
/Users/jigyasa/miniconda3/lib/python3.8/site-packages/pyscf
When PYTHONPATH is set, it is recommended to load these plugins through the environment variable PYSCF_EXT_PATH


In [5]:
# Advice on choosing the basis sets for different calculations 

#https://sites.google.com/site/orcainputlibrary/basis-sets?pli=1

In [325]:
frames=[]
for r in range(2,6):
    for n in range(3,6):
        angles = np.linspace(0.01, 2*np.pi/n-0.1, 5)
        for psi in angles:
            for z1 in range(2,5):
                for z2 in range(z1+1, 6):
                    splus, sminus = generate_nu3_degen_structs(r=r,n=n,psi=psi,z1=z1,z2=z2,center_species='Li', ring_species='Mg', z2species='H')
                    frames.append(splus)
                    frames.append(sminus)

for f in frames: 
    f.center()
# view(frames)

In [326]:
idx = [653, 652, 641, 636, 631, 621, 619, 618, 466, 454, 453, 451, 450, 444, 443, 437, 432, 428, 427, 423, 301, 
      300, 293, 279, 274, 264, 260, 255, 120, 118, 117, 116, 114, 111, 100, 98, 89, 86, 84, 77, 72, 67, 65
      ]
idx_not_conv = [629, 541, 540, 539, 538, 535, 534, 529, 528, 527, 526, 522, 517, 516, 515, 514, 511, 510, 505, 
                504, 503, 502, 493, 492, 491, 490, 487, 481, 480, 479, 478, 469, 468, 467, 457, 456, 455, 445,
                442, 433, 431, 430, 420, 409, 408, 407, 402, 397, 391, 385, 384, 373, 372, 361, 360, 349, 348,
                337, 336, 325, 324, 313, 312, 235, 234, 187, 186, 181, 180, 179, 178, 169, 168, 167, 166, 157, 
                156, 155, 154, 145, 144, 143, 142, 133, 132, 131, 130, 121, 109, 108, 105, 104, 103, 102, 97, 96,
                93, 92, 91, 90, 88, 85, 81, 80, 79, 78, 73
               ]
purge = idx + idx_not_conv
purge.sort()
purge_all = []
for i in purge: 
    if i%2==0 and i+1 not in purge: 
        purge_all.append(i)
        purge_all.append(i+1)
#         print(i, i+1, "even i, added")
    elif i%2==1 and i-1 not in purge: 
        purge_all.append(i-1)
        purge_all.append(i)
#         print(i-1, i, "odd i, added")
    else: 
        purge_all.append(i)
purge_all.reverse()
for i in purge_all: 
    frames.pop(i)

In [ ]:
en = []
for idx, f in enumerate(frames[2:]): 
    try:
        jj = pyscf_rks_calc(f)
        en.append(jj)
        print(idx+2, "done \n")
    except Exception as e: 
        print(e, idx+2, "\n")
        continue

converged SCF energy = -1207.79767639347
2 done 

converged SCF energy = -1207.7976765657
3 done 

converged SCF energy = -1207.77579124706
4 done 

converged SCF energy = -1207.77579136121
5 done 

converged SCF energy = -1207.73767133132
6 done 

converged SCF energy = -1207.73767132616
7 done 

converged SCF energy = -1207.77629071708
8 done 

converged SCF energy = -1207.77629081414
9 done 

converged SCF energy = -1207.75062998051
10 done 

converged SCF energy = -1207.75063003808
11 done 

converged SCF energy = -1207.73983085615
12 done 

converged SCF energy = -1207.73983091043
13 done 

converged SCF energy = -1207.79714773664
14 done 

converged SCF energy = -1207.79714826784
15 done 

converged SCF energy = -1207.77589524674
16 done 

converged SCF energy = -1207.77589560106
17 done 

converged SCF energy = -1207.73802120508
18 done 

converged SCF energy = -1207.73802119467
19 done 



/scratch/nigam/anaconda3/envs/py3.7/lib/python3.8/site-packages/pyscf/symm/Dmatrix.py:156: RuntimeWarning: invalid value encountered in arccos
  gamma = numpy.arccos(yp.dot(c2[1]))


converged SCF energy = -1207.77616328326
20 done 

converged SCF energy = -1207.77616358012
21 done 

converged SCF energy = -1207.75051591491
22 done 

converged SCF energy = -1207.7505160908
23 done 

converged SCF energy = -1207.7398212964
24 done 

converged SCF energy = -1207.73982146228
25 done 

converged SCF energy = -1207.79651885845
26 done 

converged SCF energy = -1207.79652203974
27 done 

converged SCF energy = -1207.77594181761
28 done 



/scratch/nigam/anaconda3/envs/py3.7/lib/python3.8/site-packages/pyscf/symm/Dmatrix.py:144: RuntimeWarning: invalid value encountered in arccos
  beta = numpy.arccos(zz)


converged SCF energy = -1207.77594391362
29 done 

converged SCF energy = -1207.73834094654
30 done 

converged SCF energy = -1207.73834082428
31 done 

converged SCF energy = -1207.77603265233
32 done 

converged SCF energy = -1207.77603447172
33 done 

converged SCF energy = -1207.75040072954
34 done 

converged SCF energy = -1207.75040179844
35 done 

converged SCF energy = -1207.73981109131
36 done 

converged SCF energy = -1207.73981213332
37 done 

converged SCF energy = -1207.79698997587
38 done 

converged SCF energy = -1207.79699703835
39 done 

converged SCF energy = -1207.7759133852
40 done 

converged SCF energy = -1207.77591799401
41 done 

converged SCF energy = -1207.73810942721
42 done 

converged SCF energy = -1207.73810901576
43 done 

converged SCF energy = -1207.77612818056
44 done 

converged SCF energy = -1207.77613223503
45 done 

converged SCF energy = -1207.75048472496
46 done 

converged SCF energy = -1207.75048712296
47 done 

converged SCF energy = -1207.739

In [327]:
len(frames)

538

In [545]:
frames[40].positions

array([[49.79903811, 49.79903811, 49.        ],
       [52.79903811, 49.79903811, 51.        ],
       [48.29903811, 52.39711432, 51.        ],
       [48.29903811, 47.20096189, 51.        ],
       [49.79903811, 52.79903811, 47.        ],
       [47.20096189, 48.29903811, 47.        ],
       [52.39711432, 48.29903811, 47.        ],
       [49.79903811, 49.79903811, 53.        ]])

In [63]:
from ase.io import write
write("bispectrum_counter_structures.xyz", frames)
np.save("bispectrum_energy.npy", en)

## feature computation

In [10]:
rascal_hypers = {
    "interaction_cutoff": 4.5,
    "cutoff_smooth_width": 0.3,
    "max_radial": 4,
    "max_angular": 2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
#     "expansion_by_species_method": "user defined",
#     "global_species": [1,6,8,7]
    
}

cg = ClebschGordanReal(5)

spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

### $\rho_{i}^{\otimes 3}$

In [23]:
rho1i = acdc_standardize_keys(rhoi)
rho1i = rho1i.keys_to_properties(['species_neighbor'])

rho2i = cg_increment(rho1i, rho1i, lcut=4, other_keys_match=["species_center"], clebsch_gordan=cg)
rho3i =  cg_increment(rho2i, rho1i, lcut=0, other_keys_match=["species_center"], clebsch_gordan=cg)

### $\rho_{i i_1 i_2}^{\otimes 1}$

In [26]:
gij =  acdc_standardize_keys(gij)
rhoii1i2_nu0 = cg_combine(gij, gij, clebsch_gordan=cg,lcut=4, other_keys_match=['species_center'])
rhoii1i2_nu1 =  cg_combine(rho1i, rhoii1i2_nu0, clebsch_gordan=cg, lcut=0, other_keys_match = ['species_center'])


### check degeneracy of $\rho_{i}^{\otimes 3}$

In [30]:
np.linalg.norm(rho3i.block(species_center = 3, inversion_sigma = 1).values[0] - rho3i.block(species_center = 3, inversion_sigma = 1).values[1])

1.680772513045776e-22

In [29]:
# rhoii1i2_nu1.keys_to_properties('species_neighbor_1')
# rhoii1i2_nu1.keys_to_properties('species_neighbor_2')

In [49]:
rhoii1i2_nu1.block(species_center = 3,species_neighbor_a=12, species_neighbor_b=12, inversion_sigma=1).samples[36:72]#values.shape


Labels([(1, 0, 1, 1), (1, 0, 1, 2), (1, 0, 1, 3), (1, 0, 1, 4),
        (1, 0, 1, 5), (1, 0, 1, 6), (1, 0, 2, 1), (1, 0, 2, 2),
        (1, 0, 2, 3), (1, 0, 2, 4), (1, 0, 2, 5), (1, 0, 2, 6),
        (1, 0, 3, 1), (1, 0, 3, 2), (1, 0, 3, 3), (1, 0, 3, 4),
        (1, 0, 3, 5), (1, 0, 3, 6), (1, 0, 4, 1), (1, 0, 4, 2),
        (1, 0, 4, 3), (1, 0, 4, 4), (1, 0, 4, 5), (1, 0, 4, 6),
        (1, 0, 5, 1), (1, 0, 5, 2), (1, 0, 5, 3), (1, 0, 5, 4),
        (1, 0, 5, 5), (1, 0, 5, 6), (1, 0, 6, 1), (1, 0, 6, 2),
        (1, 0, 6, 3), (1, 0, 6, 4), (1, 0, 6, 5), (1, 0, 6, 6)],
       dtype=[('structure', '<i4'), ('center', '<i4'), ('neighbor_1', '<i4'), ('neighbor_2', '<i4')])

In [46]:
np.linalg.norm(rhoii1i2_nu1.block(species_center = 3,species_neighbor_a=1, species_neighbor_b=12, inversion_sigma=1 ).values[:36])

2.5486800904721503e-07

In [48]:
# diff = np.sum(rhoii1i2_nu1.block(species_center = 6).values[:64], axis=0) - np.sum(rhoii1i2_nu1.block(species_center = 6).values[64:], axis=0)
diff = rhoii1i2_nu1.block(species_center = 3,species_neighbor_a=12, species_neighbor_b=12, inversion_sigma=1).values[:36] - rhoii1i2_nu1.block(species_center = 3,species_neighbor_a=12, species_neighbor_b=12, inversion_sigma=1).values[36:72]

np.linalg.norm(diff)

8.407296057023224e-08

## set up an encoder-decoder like architecture

In [319]:
import equistore
import torch
import equistore.operations as operations
from tqdm import tqdm
torch.set_default_dtype(torch.float64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  


class featDataset(torch.utils.data.Dataset):
    #Dataset class
    def __init__(self, feature_path, frames):
        #
        self.features = np.load(feature_path, mmap_mode = 'r')
        self.keys_features = equistore.io._labels_from_npz(self.features["keys"])
        self.currentkey = 0
        self.frames = frames

    def __len__(self):
        return len(self.frames)
    
    def __getitem__(self, structure_idx):
        feature_block, feature_key = self.generate_feature_block(self.features, structure_idx)        
        #samples_filter, target_block_samples = self.get_index_from_idx(self.target.block(self.currentkey).samples, structure_idx)
        structure = [frames[i] for i in structure_idx]
        #Modify feature_block to tensormap
        feature_map = TensorMap(feature_key, feature_block)
        return feature_map, structure

    
    def generate_feature_block(self, memmap, structure_idx):
        #Generate the block from npz file
        output = []
        if self.currentkey is None:
            feature_key = self.keys_features     
        else:
            feature_key = Labels(["order_nu", "inversion_sigma", "spherical_harmonics_l", "species_center", "species_neighbor_a", "species_neighbor_b"], 
                                 np.asarray(self.keys_features[self.currentkey].astype(list), dtype=np.int32).reshape(-1,6))

#             feature_key = [self.keys_features[self.currentkey]]
        for key in feature_key:
            block_index = list(self.keys_features).index(key)
            prefix = f"blocks/{block_index}/values"        
            block_samples = equistore.io._labels_from_npz(memmap[f"{prefix}/samples"])
            block_components = []
            for i in range(1):
                block_components.append(equistore.io._labels_from_npz(memmap[f"{prefix}/components/{i}"]))
            block_properties = equistore.io._labels_from_npz(memmap[f"{prefix}/properties"])
             

            samples_filter, block_samples = self.get_index_from_idx(block_samples, structure_idx)

            block_data = memmap[f"{prefix}/data"][samples_filter]
            block = TensorBlock(block_data, block_samples, block_components, block_properties)
            output.append(block)
        return output, feature_key
    
    def get_n_properties(self, memmap, key):
        block_index = list(self.keys_features).index(key)
        prefix = f"blocks/{block_index}/values"  
        block_properties = equistore.io._labels_from_npz(memmap[f"{prefix}/properties"])
        
        return len(block_properties)
    
    def get_index_from_idx(self, block_samples, structure_idx):
        #Get samples label from IDX
        samples = Labels(names = ['structure'], values = np.array(structure_idx).reshape(-1,1))
        all_samples = block_samples[['structure']].tolist()
        set_samples_to_slice = set(samples.tolist())
        samples_filter = np.array(
            [sample in set_samples_to_slice for sample in all_samples]
        )
        new_samples = block_samples[samples_filter]
        
        return samples_filter, new_samples
    
    def collate_output_values(blocks):
        feature_out = []
        for sample_output in blocks:
            feature_block, structure = sample_output
            for z in feature_block:
                feature_out.append(torch.tensor(z.values))

        return feature_out


In [315]:
def mse_loss(pred, target):
    return torch.mean((pred - target)**2)


class BlockEncoderDecoder(torch.nn.Module):
    def __init__(self, seed=123, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = torch.nn.Linear(
            in_features=kwargs["input_size"], out_features=kwargs["hidden_size"]
        )
        self.encoder_output_layer = torch.nn.Linear(
            in_features=kwargs["hidden_size"], out_features=kwargs["intermediate_size"]
        )
        self.decoder_hidden_layer = torch.nn.Linear(
            in_features=kwargs["intermediate_size"], out_features=kwargs["hidden_size"]
        )
        self.decoder_output_layer = torch.nn.Linear(
            in_features=kwargs["hidden_size"], out_features=kwargs["input_size"]
        )
        self.to(device)
        
    def forward(self, x):
        print("from block forward", x.keys)
        x = torch.from_numpy(x.block().values)
        activation = self.encoder_hidden_layer(x)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed

    
    def fit(self,traindata_loader, loss_function, optimizer_type, lr, reg, n_epochs):
        if optimizer_type == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
            threshold = 200
            scheduler_threshold = 50
            tol = 0
            history_step = 1000
        
        elif optimizer_type == "LBFGS":
            optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)
            threshold = 30
            scheduler_threshold = 10
            tol = 0
            history_step = 10
            
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, scheduler_threshold, gamma = 0.5)
        best_state = copy.deepcopy(self.state_dict())
        lowest_loss = torch.tensor(9999)
        pred_loss = torch.tensor(0)
        trigger = 0
        loss_history = []
        pbar = tqdm(range(n_epochs))
        
        for epoch in pbar:
            pbar.set_description(f"Epoch: {epoch}")
            pbar.set_postfix(pred_loss = pred_loss.item(), lowest_loss = lowest_loss.item(), trigger = trigger)
            
            for x_data, structure in traindata_loader: 
                optimizer.zero_grad()
                #x_data, y_data = x_data.to(self.device), y_data.to(self.device)
                if optimizer_type == "LBFGS":
                    def closure():
                        optimizer.zero_grad()
                        _pred = self.forward(x_data)                                        
                        _pred_loss = loss_function(_pred, x_data.block().values)
                        _pred_loss = torch.nan_to_num(_pred_loss, nan=lowest_loss.item(), posinf = lowest_loss.item(), neginf = lowest_loss.item())                 
                        _reg_loss = self.get_regression_values(reg.item()) #Only works for 1 layer
                        _new_loss = _pred_loss + _reg_loss
                        _new_loss.backward()
                        return _new_loss
                    optimizer.step(closure)

                elif optimizer_type == "Adam":
                    pred = self.forward(x_data)
#                     print(pred, x_data.block().values )
                    pred_loss = loss_function(pred, torch.from_numpy(x_data.block().values))
                    
                    #reg_loss = self.get_regression_values(reg.item())#Only works for 1 layer
                    new_loss = pred_loss #+ reg_loss
                    new_loss.backward()

                    optimizer.step()
                
            with torch.no_grad():
                current_loss = 0 
                for x_data, structure in traindata_loader:
                    pred = self.forward(x_data)
                    current_loss  += loss_function(pred, torch.from_numpy(x_data.block().values)) #Loss should be normalized already
                pred_loss = current_loss
                reg_loss = self.get_regression_values(reg.item()) 
                new_loss = pred_loss + reg_loss
                if pred_loss >100000 or (pred_loss.isnan().any()) :
                    print ("Optimizer shows weird behaviour, reinitializing at previous best_State")
                    self.load_state_dict(best_state)
                    if optimizer_type == "Adam":
                        optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
                    elif optimizer_type == "LBFGS":
                        optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)

                if epoch % history_step == 1:
                    loss_history.append(lowest_loss.item())
                
                if lowest_loss - new_loss > tol: #threshold to stop training
                    best_state = copy.deepcopy(self.state_dict())
                    lowest_loss = new_loss 
                    trigger = 0    
                    
                else:
                    trigger += 1
                    scheduler.step()
                    if trigger > threshold:
                        self.load_state_dict(best_state)
                        print ("Implemented early stopping with lowest_loss: {}".format(lowest_loss))
                        return loss_history
        return loss_history
    
    def get_regression_values(self, reg_weights):
        output = []
        for param in self.parameters():
            output.append(torch.sum(torch.pow(param,2)))
        try:
            output = torch.sum(torch.tensor(output) * reg_weights)
        except:
            output = 0
        return output


    

class ThreeCenterModel(torch.nn.ModuleDict):
    
    def __init__(self, feature, hidden_size=128, intermediate_size=18):
        super().__init__()
        self.feature = feature
        self.loss_history={}
        self.models = torch.nn.ModuleDict()
        for k, b in self.feature:
            input_size = b.values.shape[-1]
#             print(k,input_size, hidden_size, intermediate_size)
            self.models[str(k)] = BlockEncoderDecoder(input_size=input_size, hidden_size=hidden_size, intermediate_size=intermediate_size)            
        self.to(device)
    
    def forward(self, feature):
        pred_blocks =[]
        for k, block in feature:
#             print(self[str(k)])
            
            recons = self[str(k)].forward(block) # expects a tensorMap
            
            newblock = TensorBlock(
                        values=recons.reshape(block.values.shape),
                        samples=block.samples,
                        components=block.components,
                        properties= block.properties
                    )
            pred_blocks.append(newblock) 
        
        keys = feature.keys
        pred_target = TensorMap(keys, pred_blocks)
        return(pred_target)
    
    def fit(self, train_dataloader, regularization_dict, optimizer_type, n_epochs, loss_function, lr):
        #Iterates through the keys of self.model, then for each key we will fit self.model[key] with data[key]
        total = len(self.models)
        for index, key in enumerate(self.feature.keys):
            print ("Now training on Block {} of {}".format(index, total))
            train_dataloader.dataset.currentkey = index
            
            loss_history_key = self.models[str(key)].fit(train_dataloader, loss_function, optimizer_type, lr, regularization_dict[str(key)], n_epochs)

            self.loss_history[str(key)] = loss_history_key

    
    


In [316]:
regularization_dict={}
for key in rhoii1i2_nu1.keys:
    regularization_dict[str(key)] = torch.tensor(0)

In [320]:
def collate_blocks(block_tuple):
    feature_tensor_map, structure_array = block_tuple[0]
    return feature_tensor_map, structure_array

train_feature_path ="./rhoii1i2n1_400.npz"
train = featDataset(train_feature_path, frames[:400])
train_Sampler = torch.utils.data.sampler.SequentialSampler(train)
train_BSampler = torch.utils.data.sampler.BatchSampler(train_Sampler, batch_size = 500, drop_last = False)

test_feature_path ="./rhoii1i2n1_400-500.npz"
test = featDataset(train_feature_path, frames[400:500])
test_Sampler = torch.utils.data.sampler.SequentialSampler(test)
test_BSampler = torch.utils.data.sampler.BatchSampler(test_Sampler, batch_size = 500, drop_last = False)

train_loader = torch.utils.data.DataLoader(train, sampler = train_BSampler, collate_fn = collate_blocks)
test_loader = torch.utils.data.DataLoader(test, sampler = test_BSampler, collate_fn = collate_blocks)

In [317]:
# rhoii1i2_nu1.keys[
rhoii1i2_nu1.blocks_matching(species_center=3)
# ]

[9, 10, 11, 12, 13, 14, 15, 16, 17, 36, 37, 38, 39, 40, 41, 42, 43, 44]

In [334]:
# ntrain = 400
# train_tmap = operations.slice(rhoii1i2_nu1, 
#                 samples = Labels(
#             names=["structure"],
#             values=np.array(
#                 list(range(ntrain))
#             ).reshape(-1,1), )
#                 )

# test_tmap = operations.slice(rhoii1i2_nu1, 
#                 samples = Labels(
#             names=["structure"],
#             values=np.array(
#                 list(range(ntrain, 500))
#             ).reshape(-1,1), )
#                 )

# from equistore.io import save, load
# # save('./rho3i_400.npz', rh)
# save('./rhoii1i2n1_400.npz', train_tmap)
# save('./rhoii1i2n1_400-500.npz', test_tmap)

/Users/jigyasa/miniconda3/lib/python3.8/site-packages/equistore/operations/slice.py:113: UserWarning: Some TensorBlocks in the sliced TensorMap are now empty, based on your choice of samples and/or properties to slice by. The keys of the empty TensorBlocks are:
 [(3,  1, 0,  1,  1, 12) (3,  1, 0,  1,  3, 12) (3,  1, 0,  1, 12,  1)
 (3,  1, 0,  1, 12,  3) (3,  1, 0,  1, 12, 12) (3,  1, 0,  3,  1, 12)
 (3,  1, 0,  3,  3, 12) (3,  1, 0,  3, 12,  1) (3,  1, 0,  3, 12,  3)
 (3,  1, 0,  3, 12, 12) (3,  1, 0, 12,  1,  1) (3,  1, 0, 12,  1,  3)
 (3,  1, 0, 12,  1, 12) (3,  1, 0, 12,  3,  1) (3,  1, 0, 12,  3,  3)
 (3,  1, 0, 12,  3, 12) (3,  1, 0, 12, 12,  1) (3,  1, 0, 12, 12,  3)
 (3, -1, 0,  1,  1, 12) (3, -1, 0,  1,  3, 12) (3, -1, 0,  1, 12,  1)
 (3, -1, 0,  1, 12,  3) (3, -1, 0,  1, 12, 12) (3, -1, 0,  3,  1, 12)
 (3, -1, 0,  3,  3, 12) (3, -1, 0,  3, 12,  1) (3, -1, 0,  3, 12,  3)
 (3, -1, 0,  3, 12, 12) (3, -1, 0, 12,  1,  1) (3, -1, 0, 12,  1,  3)
 (3, -1, 0, 12,  1, 12) (3, -1, 0, 12

In [322]:
model = ThreeCenterModel(rhoii1i2_nu1)

In [323]:
model.fit(train_loader, regularization_dict, optimizer_type="Adam", n_epochs=20, loss_function=mse_loss, lr=0.01)

Now training on Block 0 of 54


Epoch: 1:   5%|███▋                                                                      | 1/20 [00:00<00:02,  6.64it/s, lowest_loss=0.000333, pred_loss=0.000333, trigger=0]

from block forward [(3, 1, 0, 1, 1, 1)]
from block forward [(3, 1, 0, 1, 1, 1)]
from block forward [(3, 1, 0, 1, 1, 1)]


Epoch: 2:  10%|███████▊                                                                      | 2/20 [00:00<00:02,  7.59it/s, lowest_loss=1.8e-5, pred_loss=1.8e-5, trigger=0]

from block forward [(3, 1, 0, 1, 1, 1)]
from block forward [(3, 1, 0, 1, 1, 1)]
from block forward [(3, 1, 0, 1, 1, 1)]


...

from block forward [(3, 1, 0, 12, 1, 1)]
from block forward [(3, 1, 0, 12, 1, 1)]


Epoch: 7:  35%|██████████████████████████▌                                                 | 7/20 [00:02<00:04,  2.87it/s, lowest_loss=1.92e-8, pred_loss=2.12e-8, trigger=1]

from block forward [(3, 1, 0, 12, 1, 1)]
from block forward [(3, 1, 0, 12, 1, 1)]


Epoch: 7:  35%|██████████████████████████▌                                                 | 7/20 [00:02<00:05,  2.54it/s, lowest_loss=1.92e-8, pred_loss=2.12e-8, trigger=1]


KeyboardInterrupt: 

In [336]:
for x_data, structures in test_loader:
#     print(x_data)
    pred = model(x_data)
#     print ("Test error for mse_full is {}".format(mse_full(pred, )))

SyntaxError: unexpected EOF while parsing (1137940781.py, line 4)

In [ ]:
# aa=[]
# bb=[]
# for x, y in train_loader:
#     aa.append(x)
#     bb.append(y)
# from itertools import chain
# list(chain.from_iterable(bb))==frames[:400]